# Utilisation de la mémoire persistante
Dans cet exercice, nous allons explorer comment intégrer une mémoire persistante dans un agent basé sur un LLM. Contrairement aux systèmes traditionnels qui perdent leur contexte après chaque interaction, un agent avec mémoire peut se souvenir des informations échangées précédemment, ce qui permet de créer des interactions plus naturelles et personnalisées.

## Définition des variables
Les variables sont lues depuis le fichier [.env](../../.env)

In [ ]:
from dotenv import dotenv_values
config = dotenv_values("../../.env")

llm_model = config.get('LLM_MODEL')
api_key = config.get('LLM_API_KEY')
mem0_api_key = config.get('MEM0_API_KEY')
# Uncomment for local api call
# api_base = config.get('LLM_API_URL')

## Configuration du llm sur dspy

In [ ]:
import dspy

lm = dspy.LM(llm_model, api_key=api_key)
# Uncomment for local api call
#lm = dspy.LM(llm_model, api_base=api_base, track_usage=True, temperature=1.5, max_tokens=1024)

dspy.configure_cache(
    enable_disk_cache=False,
    enable_memory_cache=False,
)
dspy.configure(lm=lm)

## Créer un client MCP Mem0

Cette section montre comment utiliser le server MCP pour intégrer la mémoire persistante. Nous allons créer des fonctions qui permettent d'ajouter, récupérer et mettre à jour les informations dans la mémoire.

In [ ]:
from fastmcp import Client

config = {
    "mcpServers": {
        "Mem0": {
            
        }
    }
}

mcpClient = Client(config)
tools = []
async with mcpClient:
    # Conversion des MCP tools en DSPy tools
    #
    print(tools)

## Créer un assistant avec mémoire persistante

Cette section définit comment utiliser la mémoire pour enrichir les interactions. L'assistant peut se souvenir des préférences et informations fournies par l'utilisateur dans ses échanges précédents.

In [ ]:
assistant = dspy.ReAct("question -> answer", tools=tools)

## Exécuter l'assistant avec mémoire

In [ ]:

import time
from rich import print

results = await assistant.acall(question="Je m'appelle Etienne daho et j'aime les burritos")
print(results.answer)
dspy.inspect_history(1)

time.sleep(60) 

results = await assistant.acall(question="Rappelle moi mon nom et mon plat préféré")
print(results.answer)
dspy.inspect_history(1)